# 0.0 Cargar dataframes

In [158]:
import pandas as pd

df = pd.read_csv('0_2_df_viajes_marzo.csv')

/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [159]:
# Estas funciones aseguran que las columnas de fechatiempo y timedelta sean de tal tipo de dato

df['tiempo_en_uso'] = pd.to_timedelta(df['tiempo_en_uso'])
df['Fecha_Arribo_datetime'] = pd.to_datetime(df['Fecha_Arribo_datetime'])
df['Fecha_Retiro_datetime'] = pd.to_datetime(df['Fecha_Retiro_datetime'])

In [160]:
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,distancia_viaje,Fecha_Retiro_datetime,Fecha_Arribo_datetime,tiempo_en_uso,velocidad
0,M,49,3307,365,01/03/2018,0:00:10,169,01/03/2018,0:13:05,2.729608,2018-03-01 00:00:10,2018-03-01 00:13:05,00:12:55,12.679471
1,F,29,9218,137,01/03/2018,0:00:13,308,01/03/2018,0:18:34,3.561729,2018-03-01 00:00:13,2018-03-01 00:18:34,00:18:21,11.645980
2,F,30,3945,54,01/03/2018,0:00:17,191,01/03/2018,0:12:13,2.019986,2018-03-01 00:00:17,2018-03-01 00:12:13,00:11:56,10.156356
3,M,25,9971,130,01/03/2018,0:00:47,271,01/03/2018,0:12:40,2.551229,2018-03-01 00:00:47,2018-03-01 00:12:40,00:11:53,12.881380
4,M,20,10551,415,01/03/2018,0:00:56,433,01/03/2018,0:05:18,1.037867,2018-03-01 00:00:56,2018-03-01 00:05:18,00:04:22,14.260767


# 1.0 Función estadisticas de Bici

In [161]:
# Notas: Esta se debe correr ya despues de que el DF de viajes tenga
## 1. Fecha_Retiro_datetime y Fecha_Arribo_datetime
## 2. distancia_recorrida
## 3. Agregar estaciones faltantes al DF_estaciones 
## 4. Remover filas con estaciones "apocrifas"

# Esta función encuentra a todas las ID de bicis, y arma un dataframe con
# estadisticas de vida y uso de cada bici

from tqdm import tqdm

def fn_bici_stats(dataframe):
    bici_unique = dataframe.Bici.unique()
    fin_lista = []
    ini_lista = []
    tiempo_usado = []
    dist_recorrida = []
    edad = []
    est_arribo = []
    est_retiro = []
    num_viajes = []
    
    for i in tqdm(range(len(bici_unique))):
        temp_dataframe = dataframe.loc[dataframe['Bici'] == bici_unique[i]]
#        temp_dataframe = temp_dataframe.reset_index(drop=True)

        inicio = min(temp_dataframe['Fecha_Retiro_datetime'])
        fin = max(temp_dataframe['Fecha_Arribo_datetime'])

   
        diferencia = fin - inicio
        
        # Si el CSV falla en mantener el formato de Timestamp, y fallan todos los otros métodos, usar esta función
        # diferencia = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S') - datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S')
        
        tiempo_en_uso = temp_dataframe.tiempo_en_uso.sum()
        
        distancia = temp_dataframe.distancia_viaje.sum()
        
        ini_lista.append(inicio)
        fin_lista.append(fin)
        tiempo_usado.append(tiempo_en_uso)
        edad.append(diferencia)
        dist_recorrida.append(distancia)
        est_arribo.append(temp_dataframe.Ciclo_Estacion_Arribo[temp_dataframe.index[-1]])
        est_retiro.append(temp_dataframe.Ciclo_Estacion_Retiro[temp_dataframe.index[0]])
        num_viajes.append(len(temp_dataframe))

        
    list_dict = {'Bici': bici_unique, 'est_retiro': est_retiro, 'est_arribo': est_arribo,
                 'fecha_inicio': ini_lista, 'fecha_fin': fin_lista, 'edad': edad, 
                 'tiempo_ciclando': tiempo_usado, 'distancia_recorrida': dist_recorrida,
                 'num_viajes': num_viajes}

    df_salida = pd.DataFrame(list_dict)
    
    return df_salida

In [163]:
# Prueba de función

from datetime import datetime
import pandas as pd

df_prueba_bici = {'Bici': [1,2,1,2,1],
                  'Ciclo_Estacion_Arribo': [3,4,5,6,7],
                  'Ciclo_Estacion_Retiro': [1,2,3,4,5],
                  'Fecha_Arribo_datetime': [datetime(2018, 3, 15, 9, 1, 30),datetime(2018, 3, 15, 10, 1, 30),
                                            datetime(2018, 3, 15, 10, 1, 30),datetime(2018, 3, 15, 11, 1, 30),
                                            datetime(2018, 3, 15, 12, 1, 30)],
                  'Fecha_Retiro_datetime': [datetime(2018, 3, 15, 8, 1, 30), datetime(2018, 3, 15, 9, 1, 30),
                                            datetime(2018, 3, 15, 9, 1, 30), datetime(2018, 3, 15, 10, 1, 30),
                                            datetime(2018, 3, 15, 11, 1, 30)],     
                  'tiempo_en_uso': [1,1,1,1,1],
                  'distancia_viaje': [1,2,1,2,2]
                 }
                
df_test_0_3 = pd.DataFrame(df_prueba_bici)

df_test_0_4 = fn_bici_stats(df_test_0_3)

df_test_0_4.head()

100%|██████████| 2/2 [00:00<00:00, 290.78it/s]


,Bici,est_retiro,est_arribo,fecha_inicio,fecha_fin,edad,tiempo_ciclando,distancia_recorrida,num_viajes
0,1,1,7,2018-03-15 08:01:30,2018-03-15 12:01:30,04:00:00,3,4,3
1,2,2,6,2018-03-15 09:01:30,2018-03-15 11:01:30,02:00:00,2,4,2


# 2.0  Correr - Estadisticas bicis

In [134]:
bike_stats = fn_bici_stats(df)

100%|██████████| 9085/9085 [08:50<00:00, 17.13it/s]


In [ ]:
bike_stats.to_csv('0_3_prueba_bike_stats.csv', index=False)

In [164]:
bike_stats

,Bici,est_retiro,est_arribo,fecha_inicio,fecha_fin,edad,tiempo_ciclando,distancia_recorrida,num_viajes
0,3307,365,25,2018-03-01 00:00:10,2018-03-06 09:07:55,5 days 09:07:45,06:04:04,52.780559,26
1,9218,137,273,2018-03-01 00:00:13,2018-03-05 07:46:09,4 days 07:45:56,03:59:21,31.467285,18
2,3945,54,141,2018-03-01 00:00:17,2018-03-06 08:40:50,5 days 08:40:33,07:21:31,52.154636,32
3,9971,130,326,2018-03-01 00:00:47,2018-03-06 09:05:22,5 days 09:04:35,10:08:37,76.285585,47
4,10551,415,305,2018-03-01 00:00:56,2018-03-06 07:58:24,5 days 07:57:28,04:08:04,34.430901,21
5,9986,39,37,2018-03-01 00:01:06,2018-03-06 09:05:51,5 days 09:04:45,08:44:47,64.921316,39
6,11030,390,294,2018-03-01 00:01:51,2018-03-06 08:15:00,5 days 08:13:09,06:02:13,63.312685,29
7,8733,64,183,2018-03-01 00:01:59,2018-03-05 21:43:31,4 days 21:41:32,06:43:17,53.275382,30
8,4190,147,304,2018-03-01 00:01:59,2018-03-06 09:08:54,5 days 09:06:55,07:30:07,64.309610,33
9,10339,151,201,2018-03-01 00:02:21,2018-03-06 08:57:37,5 days 08:55:16,07:17:48,53.831413,30
